In [ ]:
import os
import pandas as pd
from tqdm import tqdm
from ase import io
from ase.md.langevin import Langevin
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase import units
from chgnet.model.dynamics import CHGNetCalculator

# === ✅ STEP 0: Set working directory to where your folders are ===
os.chdir("/home/mehuldarak/summer")  # ← change this to your actual path!

# === STEP 1: Configuration ===
relaxed_root = "relax_final"
md_root = "md"
traj_dir = os.path.join(md_root, "mdtraj_prime")
cif_dir = os.path.join(md_root, "mdcifs_prime")
os.makedirs(traj_dir, exist_ok=True)
os.makedirs(cif_dir, exist_ok=True)

temperatures = [360, 480]
timestep = 1 * units.fs
n_steps = 1000
snapshot_interval = 5
friction = 0.01

# === STEP 2: Metadata parser ===
def parse_filename_metadata(fname):
    try:
        tokens = os.path.splitext(fname)[0].split("_")
        return dict(
            full_name=fname,
            llzo_dir=tokens[2],
            llzo_termination=tokens[3],
            llzo_order=tokens[4],
            stoichiometry=tokens[5],
            li_dir=tokens[7],
            notes=""
        )
    except Exception as e:
        return dict(full_name=fname, notes=f"parse_error: {str(e)}")

# === STEP 3: MD runner ===
def run_md_task(cif_path, base_name, temperature):
    print(f"🔥 Running MD: {base_name} @ {temperature}K")
    atoms = io.read(cif_path)
    atoms.calc = CHGNetCalculator()
    MaxwellBoltzmannDistribution(atoms, temperature_K=temperature)

    traj_path = os.path.join(traj_dir, f"{base_name}_T{temperature}.traj")
    traj_writer = io.Trajectory(traj_path, 'w', atoms)

    snapshots = []

    def save_snapshot(a=atoms):
        traj_writer.write(a)
        snapshots.append(a.copy())
        print(f"[snapshot] {len(snapshots)}", flush=True)

    dyn = Langevin(atoms, timestep, temperature_K=temperature, friction=friction)
    dyn.attach(save_snapshot, interval=snapshot_interval)
    dyn.run(n_steps)
    traj_writer.close()

    print(f"✅ Done: {base_name}_T{temperature}.traj | Snapshots: {len(snapshots)}")

    for i, snap in enumerate(snapshots):
        cif_file = f"{base_name}_T{temperature}_{i:04d}.cif"
        cif_path_out = os.path.join(cif_dir, cif_file)
        io.write(cif_path_out, snap)
        if i in [0, len(snapshots) - 1]:
            print(f"[CIF] {cif_file}")

    return dict(
        base_name=base_name,
        temperature_K=temperature,
        n_snapshots=len(snapshots),
        source_path=cif_path
    )

# === STEP 4: Main Loop
results = []
cif_jobs = []

for root, _, files in os.walk(relaxed_root):
    for fname in files:
        if fname.endswith(".cif") and fname.startswith("cellrelaxed_"):
            cif_path = os.path.join(root, fname)
            base_name = os.path.splitext(fname)[0]
            parsed = parse_filename_metadata(fname)
            for T in temperatures:
                cif_jobs.append((cif_path, base_name, T, parsed))

print(f"📦 Total jobs: {len(cif_jobs)}")

for cif_path, base_name, T, parsed in tqdm(cif_jobs):
    result = run_md_task(cif_path, base_name, T)
    result.update(parsed)
    results.append(result)

# === STEP 5: Save Excel
df = pd.DataFrame(results)
excel_path = os.path.join(md_root, "mdinfo_prime.xlsx")
df.to_excel(excel_path, index=False)

print("\n✅ All MD runs done.")
print(f"📁 Trajs → {traj_dir}")
print(f"📁 Cifs  → {cif_dir}")
print(f"📝 Info  → {excel_path}")


📦 Total jobs: 30


  0%|          | 0/30 [00:00<?, ?it/s]

🔥 Running MD: cellrelaxed_LLZO_010_Li_order4_off__Li_110_slab_heavy @ 300K
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda
[snapshot] 1
[snapshot] 2
[snapshot] 3
[snapshot] 4
[snapshot] 5
[snapshot] 6
[snapshot] 7
[snapshot] 8
[snapshot] 9
[snapshot] 10
[snapshot] 11
[snapshot] 12
[snapshot] 13
[snapshot] 14
[snapshot] 15
[snapshot] 16
[snapshot] 17
[snapshot] 18
[snapshot] 19
[snapshot] 20
[snapshot] 21
[snapshot] 22
[snapshot] 23
[snapshot] 24
[snapshot] 25
[snapshot] 26
[snapshot] 27
[snapshot] 28
[snapshot] 29
[snapshot] 30
[snapshot] 31
[snapshot] 32
[snapshot] 33
[snapshot] 34
[snapshot] 35
[snapshot] 36
[snapshot] 37
[snapshot] 38
[snapshot] 39
[snapshot] 40
[snapshot] 41
[snapshot] 42
[snapshot] 43
[snapshot] 44
[snapshot] 45
[snapshot] 46
[snapshot] 47
[snapshot] 48
[snapshot] 49
[snapshot] 50
[snapshot] 51
[snapshot] 52
[snapshot] 53
[snapshot] 54
[snapshot] 55
[snapshot] 56
[snapshot] 57
[snapshot] 58
[snapshot] 59
[snapshot] 60
[snapshot] 61
[snaps

  3%|▎         | 1/30 [04:09<2:00:41, 249.69s/it]

[CIF] cellrelaxed_LLZO_010_Li_order4_off__Li_110_slab_heavy_T300_0200.cif
🔥 Running MD: cellrelaxed_LLZO_010_Li_order4_off__Li_110_slab_heavy @ 450K
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda
[snapshot] 1
[snapshot] 2
[snapshot] 3
[snapshot] 4
[snapshot] 5
[snapshot] 6
[snapshot] 7
[snapshot] 8
[snapshot] 9
[snapshot] 10
[snapshot] 11
[snapshot] 12
[snapshot] 13
[snapshot] 14
[snapshot] 15
[snapshot] 16
[snapshot] 17
[snapshot] 18
[snapshot] 19
[snapshot] 20
[snapshot] 21
[snapshot] 22
[snapshot] 23
[snapshot] 24
[snapshot] 25
[snapshot] 26
[snapshot] 27
[snapshot] 28
[snapshot] 29
[snapshot] 30
[snapshot] 31
[snapshot] 32
[snapshot] 33
[snapshot] 34
[snapshot] 35
[snapshot] 36
[snapshot] 37
[snapshot] 38
[snapshot] 39
[snapshot] 40
[snapshot] 41
[snapshot] 42
[snapshot] 43
[snapshot] 44
[snapshot] 45
[snapshot] 46
[snapshot] 47
[snapshot] 48
[snapshot] 49
[snapshot] 50
[snapshot] 51
[snapshot] 52
[snapshot] 53
[snapshot] 54
[snapshot] 55
[snapshot] 56
[s

...

[CIF] cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T450_0200.cif
🔥 Running MD: cellrelaxed_LLZO_001_Zr_code93_sto__Li_110_slab_heavy @ 300K
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda
[snapshot] 1
[snapshot] 2
[snapshot] 3
[snapshot] 4
[snapshot] 5
[snapshot] 6
[snapshot] 7
[snapshot] 8
[snapshot] 9
[snapshot] 10
[snapshot] 11
[snapshot] 12
[snapshot] 13
[snapshot] 14
[snapshot] 15
[snapshot] 16
[snapshot] 17
[snapshot] 18
[snapshot] 19
[snapshot] 20
[snapshot] 21
[snapshot] 22
[snapshot] 23
[snapshot] 24
[snapshot] 25
[snapshot] 26
[snapshot] 27
[snapshot] 28
[snapshot] 29
[snapshot] 30
[snapshot] 31
[snapshot] 32
[snapshot] 33
[snapshot] 34
[snapshot] 35
[snapshot] 36
[snapshot] 37
[snapshot] 38
[snapshot] 39
[snapshot] 40
[snapshot] 41
[snapshot] 42
[snapshot] 43
[snapshot] 44
[snapshot] 45
[snapshot] 46
[snapshot] 47
[snapshot] 48
[snapshot] 49
[snapshot] 50
[snapshot] 51
[snapshot] 52
[snapshot] 53
[snapshot] 54
[snapshot] 55
[snapshot] 56
[s

 97%|█████████▋| 29/30 [2:48:45<05:49, 349.02s/it]

[CIF] cellrelaxed_LLZO_001_Zr_code93_sto__Li_110_slab_heavy_T300_0200.cif
🔥 Running MD: cellrelaxed_LLZO_001_Zr_code93_sto__Li_110_slab_heavy @ 450K
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda
[snapshot] 1
[snapshot] 2
[snapshot] 3
[snapshot] 4
[snapshot] 5
[snapshot] 6
[snapshot] 7
[snapshot] 8
[snapshot] 9
[snapshot] 10
[snapshot] 11
[snapshot] 12
[snapshot] 13
[snapshot] 14
[snapshot] 15
[snapshot] 16
[snapshot] 17
[snapshot] 18
[snapshot] 19
[snapshot] 20
[snapshot] 21
[snapshot] 22
[snapshot] 23
[snapshot] 24
[snapshot] 25
[snapshot] 26
[snapshot] 27
[snapshot] 28
[snapshot] 29
[snapshot] 30
[snapshot] 31
[snapshot] 32
[snapshot] 33
[snapshot] 34
[snapshot] 35
[snapshot] 36
[snapshot] 37
[snapshot] 38
[snapshot] 39
[snapshot] 40
[snapshot] 41
[snapshot] 42
[snapshot] 43
[snapshot] 44
[snapshot] 45
[snapshot] 46
[snapshot] 47
[snapshot] 48
[snapshot] 49
[snapshot] 50
[snapshot] 51
[snapshot] 52
[snapshot] 53
[snapshot] 54
[snapshot] 55
[snapshot] 56
[s

100%|██████████| 30/30 [2:57:49<00:00, 355.65s/it]

[CIF] cellrelaxed_LLZO_001_Zr_code93_sto__Li_110_slab_heavy_T450_0200.cif

✅ All MD runs done.
📁 Trajs → md/mdtraj
📁 Cifs  → md/mdcifs
📝 Info  → md/mdinfo.xlsx


In [1]:
!pip install chgnet

  Preparing metadata (setup.py) ... done
  Using cached torch-2.7.1-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Preparing metadata (setup.py) ... done
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.6.77-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.6.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.6.80-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.5.1.17-py3-none-manylinux_2_28_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.6.4.1-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.3.0.4-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.7.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metada

In [6]:
import os
import pandas as pd
from random import sample
from ase import io
from ase.io import read, write
from pymatgen.io.ase import AseAtomsAdaptor

# === Config ===
input_dir = "/home/phanim/harshitrawat/mehul/summer/md/mdcifs"
cif_out_dir = "/home/phanim/harshitrawat/mehul/summer/md/mdcifs_strained_perturbed"
traj_out_dir = "/home/phanim/harshitrawat/mehul/summer/md/mdtraj_strained_perturbed"
os.makedirs(cif_out_dir, exist_ok=True)
os.makedirs(traj_out_dir, exist_ok=True)

strain_percents = [-0.03, -0.02, 0.02, 0.03]
perturb_amplitude = 0.01
sample_size = 250

# === Sample 250 CIFs ===
all_cifs = [f for f in os.listdir(input_dir) if f.endswith(".cif")]
selected = sample(all_cifs, sample_size)
metadata = []

print(f"🚀 Applying strain + perturbation to {sample_size} CIFs...")

for i, fname in enumerate(selected, 1):
    atoms = read(os.path.join(input_dir, fname))
    base = fname[:-4]

    for eps in strain_percents:
        strained = atoms.copy()
        strained.set_cell(strained.cell * (1 + eps), scale_atoms=True)

        # Convert to pymatgen and apply perturbation
        struct = AseAtomsAdaptor.get_structure(strained)
        struct.perturb(perturb_amplitude)

        # Back to ASE
        perturbed = AseAtomsAdaptor.get_atoms(struct)

        # Save CIF
        cif_name = f"{base}_strain{int(eps*100):+d}_perturbed.cif"
        io.write(os.path.join(cif_out_dir, cif_name), perturbed)

        # Save TRAJ
        traj_name = cif_name.replace(".cif", ".traj")
        io.write(os.path.join(traj_out_dir, traj_name), perturbed)

        metadata.append({
            "original_file": fname,
            "strained_file": cif_name,
            "traj_file": traj_name,
            "strain_percent": eps,
            "perturb_amplitude": perturb_amplitude
        })

    print(f"[{i}/{sample_size}] ✅ {fname}")

# === Save metadata
excel_path = "md/strain_perturb_info.xlsx"
pd.DataFrame(metadata).to_excel(excel_path, index=False)

print("\n✅ Done: All strained + perturbed structures saved.")
print(f"📁 CIFs  → {cif_out_dir}")
print(f"📁 TRAJs → {traj_out_dir}")
print(f"🧾 Info  → {excel_path}")


🚀 Applying strain + perturbation to 250 CIFs...
[1/250] ✅ cellrelaxed_LLZO_001_Zr_code93_sto__Li_110_slab_heavy_T300_0176.cif
[2/250] ✅ cellrelaxed_LLZO_010_La_order0_off__Li_110_slab_heavy_T300_0161.cif
[3/250] ✅ cellrelaxed_LLZO_010_O_order5_off__Li_110_slab_heavy_T450_0055.cif
[4/250] ✅ cellrelaxed_LLZO_010_Li_order4_off__Li_100_slab_heavy_T450_0110.cif
[5/250] ✅ cellrelaxed_LLZO_001_Zr_code93_sto__Li_111_slab_heavy_T300_0167.cif
[6/250] ✅ cellrelaxed_LLZO_010_Li_order4_off__Li_111_slab_heavy_T450_0115.cif
[7/250] ✅ cellrelaxed_LLZO_001_Zr_code93_sto__Li_111_slab_heavy_T450_0004.cif
[8/250] ✅ cellrelaxed_LLZO_001_Zr_code93_sto__Li_111_slab_heavy_T450_0158.cif
[9/250] ✅ cellrelaxed_LLZO_010_Li_order0_off__Li_111_slab_heavy_T450_0016.cif
[10/250] ✅ cellrelaxed_LLZO_010_Li_order0_off__Li_100_slab_heavy_T300_0069.cif
[11/250] ✅ cellrelaxed_LLZO_010_O_order5_off__Li_110_slab_heavy_T450_0096.cif
[12/250] ✅ cellrelaxed_LLZO_010_O_order5_off__Li_110_slab_heavy_T450_0090.cif
[13/250] ✅ cellr

ModuleNotFoundError: No module named 'openpyxl'

In [8]:
pd.DataFrame(metadata).to_excel(excel_path, index=False)

print("\n✅ Done: All strained + perturbed structures saved.")
print(f"📁 CIFs  → {cif_out_dir}")
print(f"📁 TRAJs → {traj_out_dir}")
print(f"🧾 Info  → {excel_path}")



✅ Done: All strained + perturbed structures saved.
📁 CIFs  → /home/phanim/harshitrawat/mehul/summer/md/mdcifs_strained_perturbed
📁 TRAJs → /home/phanim/harshitrawat/mehul/summer/md/mdtraj_strained_perturbed
🧾 Info  → md/strain_perturb_info.xlsx


In [ ]:
import os
import pandas as pd
import numpy as np
from ase.io import read
from ase.calculators.calculator import CalculatorError
from mace.calculators import MACECalculator

# === Config ===
folder = "/home/mehuldarak/summer/md/mdcifs_strained_perturbed"
output_excel = "/home/mehuldarak/summer/md/mdinfo_mace_predictions_matrix.xlsx"
model_path = "/home/mehuldarak/summer/mace_models/universal/2024-01-07-mace-128-L2_epoch-199.model"

# === Load custom MACE model on GPU 0 ===
calc = MACECalculator(model_path=model_path, device="cuda:0")
print("✅ Loaded MACE model on cuda:0")
def extract_info_from_cif(cif_path):
    try:
        atoms = read(cif_path)
        atoms.calc = calc
        energy = atoms.get_potential_energy()
        forces = atoms.get_forces()
        fmax = np.max(np.linalg.norm(forces, axis=1))
        stress = atoms.get_stress(voigt=False).tolist()
        magmom = atoms.get_magnetic_moment() if "magmom" in atoms.arrays else None

        return {
            "file": os.path.basename(cif_path),
            "energy_eV": energy,
            "fmax_eV_per_A": fmax,
            "stress_tensor": stress,
            "magmom_total": magmom
        }
    except (CalculatorError, Exception) as e:
        return {"file": os.path.basename(cif_path), "error": str(e)}

results = []
for fname in sorted(os.listdir(folder)):
    if fname.endswith(".cif"):
        path = os.path.join(folder, fname)
        result = extract_info_from_cif(path)
        results.append(result)
        print(f"✅ {fname}" if "error" not in result else f"❌ {fname} — {result['error']}")

pd.DataFrame(results).to_excel(output_excel, index=False)
print(f"\n🧾 Saved predictions to: {output_excel}")


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/mace/calculators/mace.py:143: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_lo

Using head Default out of ['Default']
No dtype selected, switching to float64 to match model dtype.
✅ Loaded MACE model on cuda:0
✅ cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0022_strain+2_perturbed.cif
✅ cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0022_strain+3_perturbed.cif
✅ cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0022_strain-2_perturbed.cif
✅ cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0022_strain-3_perturbed.cif
✅ cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0039_strain+2_perturbed.cif
✅ cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0039_strain+3_perturbed.cif
✅ cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0039_strain-2_perturbed.cif
✅ cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0039_strain-3_perturbed.cif
✅ cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0043_strain+2_perturbed.cif
✅ cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0043_str

In [5]:
import os
import json
import pandas as pd
import numpy as np
from ase.io import read
from ase.calculators.calculator import CalculatorError
from chgnet.model.dynamics import CHGNetCalculator
import torch

# === User Config ===
cif_path = "/home/phanim/harshitrawat/summer/summer_llzo_cifs/LLZO_010_La_order0_off.cif"  # 🔁 Replace with actual CIF path
output_excel = "chgnet_single_test_output.xlsx"

# === Setup Calculator ===
calc = CHGNetCalculator(device="cuda")  # or "cpu"
calc.model.to("cuda" if torch.cuda.is_available() else "cpu")
print("🔧 CHGNet model loaded on:", next(calc.model.parameters()).device)

# === Extraction Function ===
def extract_info_from_cif(path):
    try:
        atoms = read(path)
        atoms.calc = calc
        energy = atoms.get_potential_energy()
        forces = atoms.get_forces()
        stress = atoms.get_stress(voigt=False).tolist()
        magmom = atoms.get_magnetic_moment() if "magmom" in atoms.arrays else None

        return {
            "file": os.path.basename(path),
            "energy_eV": energy,
            "forces_per_atom_eV_per_A": json.dumps(forces.tolist()),
            "stress_tensor": json.dumps(stress),
            "magmom_total": magmom,
            "natoms": len(atoms),
        }
    except (CalculatorError, Exception) as e:
        return {"file": os.path.basename(path), "error": str(e)}

# === Run ===
result = extract_info_from_cif(cif_path)
df = pd.DataFrame([result])
df.to_excel(output_excel, index=False)

print("\n✅ Done. Results written to:", output_excel)


CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cpu
🔧 CHGNet model loaded on: cpu

✅ Done. Results written to: chgnet_single_test_output.xlsx


In [4]:
!pip install openpyxl

In [ ]:
import mace
print(mace.__version)

In [1]:
# test_distributed_mig.py
import torch
import os

print(f"[PID {os.getpid()}] CUDA_VISIBLE_DEVICES: {os.environ['CUDA_VISIBLE_DEVICES']}")
print(f"[PID {os.getpid()}] torch.cuda.device_count(): {torch.cuda.device_count()}")

for i in range(torch.cuda.device_count()):
    print(f"[PID {os.getpid()}] Device {i}: {torch.cuda.get_device_name(i)}")


[PID 1428704] CUDA_VISIBLE_DEVICES: 0,1
[PID 1428704] torch.cuda.device_count(): 2


DeferredCudaCallError: CUDA call failed lazily at initialization with error: device >= 0 && device < num_gpus INTERNAL ASSERT FAILED at "/pytorch/aten/src/ATen/cuda/CUDAContext.cpp":52, please report a bug to PyTorch. device=1, num_gpus=1

CUDA call was originally invoked at:

  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/tornado/platform/asyncio.py", line 211, in start
    self.asyncio_loop.run_forever()
  File "/hom